Based on the a json file downloaded from 
https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php

Plot location of earthquackes in April 1st 2020

### Crash Course Text

In [1]:
import json

In [2]:
# open json file
filename = "eq_1_day_m1.json"

with open(filename) as f:
    all_eq_data = json.load(f)

readable_file = "eq_1_day_m1.json"

with open(readable_file, "w") as f:
    json.dump(all_eq_data, f, indent=4)

In [3]:
# subset features in a list of dicts
all_eq_dicts = all_eq_data["features"]

# count how many earthquakes happened on Apr 1st 2020 in the world
print(len(all_eq_dicts))

228


In [4]:
# extract the mag
mags =  []
for eq_dict in all_eq_dicts:
    mag = eq_dict["properties"]["mag"]
    mags.append(mag)
    
mags[:10]

[1.7, 2.55, 1.1, 3.1, 2.78, 5.2, 4.7, 1.47, 1, 2.99]

In [5]:
# alternative: list comprehension
# list comprehension


mags_ls= [eq_dict["properties"]["mag"] for eq_dict in all_eq_dicts]

mags_ls[:10]

[1.7, 2.55, 1.1, 3.1, 2.78, 5.2, 4.7, 1.47, 1, 2.99]

In [6]:
# apply to extract list of mags, lat and lon

mags = [eq_dict["properties"]["mag"] for eq_dict in all_eq_dicts]
lons = [eq_dict["geometry"]["coordinates"][0] for eq_dict in all_eq_dicts]
lats = [eq_dict["geometry"]["coordinates"][1] for eq_dict in all_eq_dicts]
locs = [eq_dict["properties"]["title"] for eq_dict in all_eq_dicts]

print(lons[:5], lats[:5])

[-117.5563333, -66.8188, -148.5198, -115.2441, -98.3073349] [35.7105, 17.9736, 64.7286, 44.4472, 34.54284286]


In [7]:
# Build a worlmap
import matplotlib.pyplot as plt
from plotly.graph_objs import Scattergeo, Layout
from plotly import offline

In [8]:
# plot map offline in a html
data = [Scattergeo(lon=lons, lat=lats)]
my_layout = Layout(title="Global Earthquakes")

fig ={"data":data, "layout":my_layout}
offline.plot(fig, filename = "global_earthquakes.html")

'global_earthquakes.html'

In [9]:
# re-arrange the data, customise the value

data = [{
    "type": "scattergeo",
    "lon": lons,
    "lat": lats,
    "text": locs,
    "marker": {
        "size":[5*mag for mag in mags],
        "color": mags,
        "colorscale": "Viridis",
        "reversescale": True,
        "colorbar": {"title": "Magnitudes"}
    }
}]


fig ={"data":data, "layout":my_layout}
offline.plot(fig, filename = "global_earthquakes.html")

'global_earthquakes.html'

### set the json file into dataframe pandas

In [10]:
import pandas as pd


In [11]:
pd.DataFrame(all_eq_dicts)

,type,properties,geometry,id
0,Feature,"{'mag': 1.7, 'place': '15km NE of Ridgecrest, ...","{'type': 'Point', 'coordinates': [-117.5563333...",ci39124527
1,Feature,"{'mag': 2.55, 'place': '2km S of Indios, Puert...","{'type': 'Point', 'coordinates': [-66.8188, 17...",pr2020093021
2,Feature,"{'mag': 1.1, 'place': '27km WSW of Ester, Alas...","{'type': 'Point', 'coordinates': [-148.5198, 6...",ak02049zlglh
3,Feature,"{'mag': 3.1, 'place': '63km E of Cascade, Idah...","{'type': 'Point', 'coordinates': [-115.2441, 4...",us70008lgl
4,Feature,"{'mag': 2.78, 'place': '9km NE of Geronimo, Ok...","{'type': 'Point', 'coordinates': [-98.3073349,...",ok2020gmxa
...,...,...,...,...
223,Feature,"{'mag': 5.3, 'place': '78km ENE of Qumong, Chi...","{'type': 'Point', 'coordinates': [98.9159, 33....",us70008kd8
224,Feature,"{'mag': 3, 'place': '65km E of Cascade, Idaho'...","{'type': 'Point', 'coordinates': [-115.2325, 4...",us70008kcy
225,Feature,"{'mag': 2.62, 'place': '2km E of Maria Antonia...","{'type': 'Point', 'coordinates': [-66.8651, 17...",pr2020092020
226,Feature,"{'mag': 1.9, 'place': '83km WSW of Larsen Bay,...","{'type': 'Point', 'coordinates': [-155.1244, 5...",ak02048cndn5


In [12]:
all_eq_dicts[0]["properties"]["mag"]

1.7

In [13]:
all_eq = pd.DataFrame(
    {'Magnitude': [eq_dict["properties"]["mag"] for eq_dict in all_eq_dicts],
     'Longitude': [eq_dict["geometry"]["coordinates"][0] for eq_dict in all_eq_dicts],
     'Latitude': [eq_dict["geometry"]["coordinates"][1] for eq_dict in all_eq_dicts],
     "Location": [eq_dict["properties"]["title"] for eq_dict in all_eq_dicts]
    })

all_eq

,Magnitude,Longitude,Latitude,Location
0,1.70,-117.556333,35.710500,"M 1.7 - 15km NE of Ridgecrest, CA"
1,2.55,-66.818800,17.973600,"M 2.6 - 2km S of Indios, Puerto Rico"
2,1.10,-148.519800,64.728600,"M 1.1 - 27km WSW of Ester, Alaska"
3,3.10,-115.244100,44.447200,"M 3.1 - 63km E of Cascade, Idaho"
4,2.78,-98.307335,34.542843,"M 2.8 - 9km NE of Geronimo, Oklahoma"
...,...,...,...,...
223,5.30,98.915900,33.124400,"M 5.3 - 78km ENE of Qumong, China"
224,3.00,-115.232500,44.412800,"M 3.0 - 65km E of Cascade, Idaho"
225,2.62,-66.865100,17.982600,"M 2.6 - 2km E of Maria Antonia, Puerto Rico"
226,1.90,-155.124400,57.119600,"M 1.9 - 83km WSW of Larsen Bay, Alaska"
